# Display orbits over the Fermi Surface

In [2]:
import numpy as np
from numpy import cos, sin, pi
from skimage import measure
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from color_maps_plotly import *
from band_structure import *
from diff_equation import *

In [3]:
a = 1
b = 1
c = 1

t   =  1.
tp  = -0.14 * t
tpp =  0.07 * t
tz  =  0.07 * t
mu  = 1.1 * t # van Hove 0.84

tau = 25

## Magnetic field
B_amp = 0.05
B_theta = 30 * pi / 180
B_phi = 0 * pi / 180

band_parameters = np.array([a, b, c, mu, t, tp, tpp, tz])

mesh_xy = 56 # 28 must be a multiple of 4
mesh_z = 11 # 11 ideal to be fast and accurate

Calculate initial kf

In [4]:
kf, vf, dkf, number_contours = discretize_FS(band_parameters, mesh_xy, mesh_z, half_FS_z = False)

Calculates kft

In [5]:
tmax = 10 * tau
dt = tmax / 500
t = np.arange(0, tmax, dt)

## Compute B ////#
B = B_func(B_amp, B_theta, B_phi)

## Run solver ///#
kft = rgk4_algorithm(kf, t, B, band_parameters)

Calculate the 3D surface

In [6]:
mesh_xy_graph = 100
mesh_z_graph = 30
kx_a = np.linspace(-pi/a, pi/a, mesh_xy_graph)
ky_a = np.linspace(-pi/b, pi/b, mesh_xy_graph)
kz_a = np.linspace(-2*pi/c, 2*pi/c, mesh_z_graph) # 2*pi/c because bodycentered unit cell

kxx, kyy, kzz = np.meshgrid(kx_a, ky_a, kz_a)

bands = e_3D_func(kxx, kyy, kzz, band_parameters)

vertices, simplices = measure.marching_cubes_classic(bands, 0)
kxf = (vertices[:,0]/(mesh_xy_graph-1)-0.5)*2*pi/a
kyf = (vertices[:,1]/(mesh_xy_graph-1)-0.5)*2*pi/b
kzf = (vertices[:,2]/(mesh_z_graph-1)-0.5)*4*pi/c

def vzFunction(kx,ky,kz):
    return abs(v_3D_func(kx, ky, kz, band_parameters)[2])

In [7]:
magma=magma_colormap()

colorscale=['rgb(20,29,67)',
           'rgb(28,76,96)',
           'rgb(16,125,121)',
           'rgb(92,166,133)',
           'rgb(182,202,175)',
           'rgb(253,245,243)',
           'rgb(230,183,162)',
           'rgb(211,118,105)',
           'rgb(174,63,95)',
           'rgb(116,25,93)',
           'rgb(51,13,53)']

fermi_surface = ff.create_trisurf(x=kxf,
                        y=kyf, 
                        z=kzf, 
                        plot_edges=False,
                        color_func = vzFunction,
                        colormap = ['rgb(174,43,85)', 'rgb(174,123,145)'],
                        simplices=simplices,
                        title="Fermi Surface")

In [ ]:
from ipywidgets import interactive, HBox, VBox, interact


trajectory = go.Scatter3d(
    x=kft[0,int(mesh_xy*mesh_z/2),:],
    y=kft[1,int(mesh_xy*mesh_z/2),:],
    z=kft[2,int(mesh_xy*mesh_z/2),:],
    mode='markers',
    marker=dict(
        color='rgb(0, 0, 255)',
        size=1,
        symbol='circle',
        line=dict(
                color='rgb(0, 0, 255)',
                width=1
                  ),
                )
)

camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.25, y=1.25, z=1.05)
)

data_to_plot = [fermi_surface.data[0], fermi_surface.data[1], trajectory]


f = go.FigureWidget(
        data = data_to_plot,
        layout = go.Layout(
            title='Fermi Surface',
        #     scene=dict(camera=camera),
            autosize=False,
            scene = dict(aspectmode = 'manual', aspectratio = dict(x=1, y=1, z=1),
                        xaxis = dict(nticks=4, range = [-pi/a,pi/a], title = 'kx'),
                        yaxis = dict(nticks=4, range = [-pi/b,pi/b], title = 'ky'),
                        zaxis = dict(nticks=4, range = [-2*pi/c,2*pi/c], title = 'kz'),
                         camera = camera
                        ),
            width=800,
            height=600,
            margin=dict(l=25,r=20,b=25,t=90)
                           ),
                    )

def update_trajectory(position):
    f.data[2].x = kft[0,position,:]
    f.data[2].y = kft[1,position,:]
    f.data[2].z = kft[2,position,:]

    
position_slider = interactive(update_trajectory, position = (0, mesh_xy*mesh_z, 10))
vb = VBox((f, position_slider))
vb.layout.align_items = 'center'
vb

    'data': [{'facecolor': array(['rgb(174, 54, 94)', 'rgb(174, 50, 91)', 'rgb(1…

In [9]:
# py.iplot([fermi_surface.data[0], fermi_surface.data[1], trajectory], filename = "myfig.pdf")